In [33]:
import pandas as pd
from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import string

In [2]:
dfb = pd.read_json('data/business.json', lines=True)
dfr = pd.read_json('data/review.json', lines=True)

In [7]:
dfb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192609 entries, 0 to 192608
Data columns (total 14 columns):
address         192609 non-null object
attributes      163773 non-null object
business_id     192609 non-null object
categories      192127 non-null object
city            192609 non-null object
hours           147779 non-null object
is_open         192609 non-null int64
latitude        192609 non-null float64
longitude       192609 non-null float64
name            192609 non-null object
postal_code     192609 non-null object
review_count    192609 non-null int64
stars           192609 non-null float64
state           192609 non-null object
dtypes: float64(3), int64(2), object(9)
memory usage: 20.6+ MB


In [3]:
dfb.head(1)

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,2818 E Camino Acequia Drive,{'GoodForKids': 'False'},1SWheh84yJXfytovILXOAQ,"Golf, Active Life",Phoenix,None,0,33.522143,-112.018481,Arizona Biltmore Golf Club,85016,5,3.0,AZ


In [8]:
relevant_categories = set(['Restaurants','Food','Nightlife','Coffee & Tea'])
def check_relevance(ctgs):
    if isinstance(ctgs, str):
        ctgs = set(ctgs.split(', '))
        if ctgs.intersection(relevant_categories):
            return 1
        else:
            return 0
    else:
        return 0

In [9]:
dfb['relevance'] = dfb['categories'].apply(check_relevance)

In [10]:
print(len(dfb))
dfb = dfb[dfb.relevance == 1]
b_ids = set(dfb.business_id)
print(len(dfb))

192609
78595 78595


In [11]:
print(len(dfr))
dfr = dfr[dfr.business_id.isin(b_ids)]
print(len(dfr))

4747892

In [13]:
dfr.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4747892 entries, 3 to 6685899
Data columns (total 9 columns):
business_id    4747892 non-null object
cool           4747892 non-null int64
date           4747892 non-null datetime64[ns]
funny          4747892 non-null int64
review_id      4747892 non-null object
stars          4747892 non-null int64
text           4747892 non-null object
useful         4747892 non-null int64
user_id        4747892 non-null object
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 362.2+ MB


In [47]:
stop_words = set(stopwords.words('english'))
punct = list(string.punctuation)

def clean_text(text):
    if isinstance(text, str):
        for ch in punct:
            text = text.replace(ch, '')
        word_tokens = text.lower().split() 
        clean_text = [w for w in word_tokens if not w in stop_words]
        ct = ' '.join(clean_text)
        return ct
    else:
        return None

In [57]:
dfr['clean_text'] = dfr['text'].progress_apply(clean_text)

In [59]:
dfr.stars.value_counts()

5.0    1883004
4.0    1234122
3.0     627507
1.0     568413
2.0     434846
Name: stars, dtype: int64

In [60]:
dfr = dfr.drop(['user_id','review_id','text'],axis=1)
print(len(dfr))
dfr = dfr.dropna(subset=['business_id', 'stars','clean_text'])
print(len(dfr))

4747892
4747892


In [61]:
dfr['sentiment'] = dfr['stars'].progress_apply(lambda s: 1 if s in [5,4] else 0)

In [63]:
dfr.to_csv('data/YelpReview.csv',index=False)

In [64]:
dfb.to_csv('data/YelpBusiness.csv',index=False)

In [65]:
dfr

,business_id,cool,date,funny,stars,useful,clean_text,sentiment
3,ikCg8xy5JIg_NGPx-MSIDA,0.0,2018-01-09 20:56:38,0.0,5.0,0.0,went lunch steak sandwich delicious caesar sal...,1
5,eU_713ec6fTGNO4BegRaww,0.0,2013-01-20 13:25:59,0.0,4.0,0.0,ill first admit excited going la tavolta food ...,1
6,3fw2X5bZYeW9xCz_zGhOHg,5.0,2016-05-07 01:21:02,4.0,3.0,5.0,tracy dessert big name hong kong one first mar...,0
7,zvO-PJCpNk4fgAVUnExYAA,1.0,2010-10-05 19:12:35,1.0,1.0,3.0,place gone hill clearly cut back staff food qu...,0
8,b2jN2mm9Wf3RcrZCgfo1cg,0.0,2015-01-18 14:04:18,0.0,2.0,1.0,really looking forward visiting beers man owar...,0
10,8mIrX_LrOnAqWsB5JrOojQ,0.0,2011-11-30 02:11:15,0.0,4.0,0.0,like walking back time every saturday morning ...,1
11,mRUVMJkUGxrByzMQ2MuOpA,0.0,2017-12-15 23:27:08,1.0,1.0,0.0,walked around 4 friday afternoon sat table bar...,0
12,FxLfqxdYPA6Z85PFKaqLrg,0.0,2016-05-07 01:36:53,0.0,4.0,0.0,wow surprised one two star reviews started ten...,1
13,LUN6swQYa4xJKaM_UEUOEw,0.0,2018-04-27 20:25:26,0.0,4.0,0.0,michael red carpet vip amazing reached needed ...,1
14,AakkkTuGZA2KBodKi2_u8A,0.0,2012-07-16 00:37:14,1.0,1.0,1.0,cannot believe things changed 3 years picked d...,0
